# Edex __: Interpretable ML

## Problem Statement and Motivations
In this edex, we'll be revisiting the gas permeability dataset to identify the physical underpinnings of gas transport through polymer membranes.

#### Goal: 
Implement several techniques to measure feature importance, identifying top chemical substructures for gas-specific polymer membrane design.

#### Motivation: 
Recall in edex 2, we created regression models to predict gas permeability from polymer structures, drastically speeding up material screening for separations. We now wish to dig into those models and identify any mechanistic insight ML might offer by means of isolating substructure-level contributions to permeability. Not only would this further inform material design, it would implicitly identify top performing polymer families for certain applications.\

With models readily available now to predict properties, can we be sure they're trustworthy? This module will delve into a few basic tools to assess what models are paying attention to and double check validity as we go. Interpretability is quickly becoming a hot topic, and it's especially necessary for scientific applications. Small checks up front can save some trouble down the line, as no one wants to commit to a model only to find out it's completely misinformed too late.



#### Papers:

Background:\
[1]	Barnett, J. Wesley, et al. “Designing Exceptional Gas-Separation Polymer Membranes Using Machine Learning.” Science Advances, vol. 6, no. 20, 2020, doi:10.1126/sciadv.aaz4301.

Permutation Importance:\
[2] https://scikit-learn.org/stable/modules/permutation_importance.html

Shapley Values:\
[3] Shapley sampling values: Strumbelj, Erik, and Igor Kononenko. "Explaining prediction models and individual predictions with feature contributions." Knowledge and information systems 41.3 (2014): 647-665.

## Framing the Problem in ML

[Put something here]

### Import libraries

Begin by importing the necessary modules. We'll be taking most of them from the old module, so you should only need to install a couple.

In [ ]:
import pandas as pd
import numpy as np
import copy
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle as pkl
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap

## Data

For convenience, the dataset for CO2 has been pickled in both RDKit fingerprint and MACCS formats. Unpack the files as follows:

In [ ]:
with open(r"MACCS_O2_Xtrain.pkl", "rb") as input_file:
    X_train = pkl.load(input_file)
with open(r"MACCS_O2_Xtest.pkl", "rb") as input_file:
    X_test = pkl.load(input_file)
with open(r"O2_ytrain.pkl", "rb") as input_file:
    y_train = pkl.load(input_file)
with open(r"O2_ytest.pkl", "rb") as input_file:
    y_test = pkl.load(input_file)

In [ ]:
print('X_train shape is ' + str(X_train.shape))
print('y_train shape is ' + str(y_train.shape))
print('X_test shape is ' + str(X_test.shape))
print('y_test shape is ' + str(y_test.shape))

Array sizes match up, so we're good to go. 

## Example: MDI and Permutation Importance
Let's start with calculating importance by permutation. This one is a handy tool from scikit-learn. The idea here is to shuffle the value of a single feature and measure how impactful it is on the overall model's performance. Here, we'll just use the correlation coefficient from before as a stand-in for that. For a vital feature, we expect that permuting the value of the feature would also permute the model's score due to heavy reliance. Conversely, unimportant features being permuted shouldn't influence the model too much since it's not that keen on using the feature. First, let's build up a random forest model called `rfr`. We're not going for extreme accuracy here, so the scores aren't too important.

#### <font color='RED'>YOUR SOLUTION:</font> 

First, let's see what the model's intrinsic feature importance looks like. Any scikit-learn random forest model will automatically calculate the decrease in impurity of each leaf node following a split based on the value of a certain feature. For example, if index 3 of our vector is massively important to the prediction, using this as the criterion to split should yield two much more self-similar leaf nodes, and therefore have a high Mean Decrease of Impurity (MDI). This is trivial to call. Note that we're using MACCS keys here, so the x-axis corresponds directly to the key number. We can read off which chemical groups these keys represent from the following page:
https://github.com/rdkit/rdkit/blob/master/rdkit/Chem/MACCSkeys.py

In [ ]:
importances = rfr.feature_importances_

fig, ax = plt.subplots()
ax.bar(range(len(importances)), importances, color='blue')
ax.set_xlabel('MACCS Key Number')
ax.set_ylabel('Normalized Importance')
ax.grid()

Looks like a particular feature is dominating prediction here. What specific chemistry is it? Why might it have so much control over the model output?

Now let's try permutation importance. Refer to the example on the scikit-learn page here: https://scikit-learn.org/stable/modules/permutation_importance.html
Use the same model, 10 permutations, and only the training dataset to start with.

#### <font color='RED'>YOUR SOLUTION:</font> 

Now plot a bar chart of your importances like we did above.

#### <font color='RED'>YOUR SOLUTION:</font> 

Does this agree or disagree with the former result? 

# Shapley Values

Now let's move on to Shapley Value analysis. This is a similar idea to permutation importance, but with a slight tweak. The SHAP procedure will iteratively sample your input space with one feature held constant at some "background" (i.e. non-contributing) value and measure the change on your output variable when that feature is fixed. This allows direct calculations of feature contributions to the actual variable of interest. The only thing to keep in mind here is that, while it may seem appealing to directly jump to cause-and-effect explanations, this is still correlation and not quite causation. To build your SHAP explainer and get values, refer to the documentation here: https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Basic%20SHAP%20Interaction%20Value%20Example%20in%20XGBoost.html\

They use an XGBoost model, but the execution on the SHAP side is exactly the same for random forest.

#### <font color='RED'>YOUR SOLUTION:</font> 

Now try making a summary plot as in their example to get a breakdown of most important features

#### <font color='RED'>YOUR SOLUTION:</font> 

This plot gives you three pieces of information: the features themselves, their SHAP values, and intuition for their impact. Note that blue points indicate that feature being a lower value, and the SHAP value indicates how it impacts the target variable (permeability, in our case). Look at the SHAP values for some of the features highlighted in the above examples. Does this agree or disagree? Recalling that these results are still only correlative, how might we use this information in our study of membranes?

# Try using one of these tools on another model from the course. 
### Explore the SHAP page documentation to see what versions you might need. Attach your findings below:

#### <font color='RED'>YOUR SOLUTION:</font> 